In [1]:
### replicate results of EVmutation with the PABP_YEAST dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from model import CouplingsModel
import tools
import scipy
from pathlib import Path
from collections import OrderedDict
# biopython SeqIO
from Bio import SeqIO
from sklearn.metrics import roc_auc_score
import scipy
from collections import OrderedDict

In [2]:
ALPHABET_PROTEIN = '-ACDEFGHIKLMNPQRSTVWY'

In [3]:
# helper functions
def encode(seqs, alphabet=ALPHABET_PROTEIN):
    '''
    Go from letters to numbers
    '''
    aa_to_i = OrderedDict((aa, i) for i, aa in enumerate( alphabet ))
    X = np.asarray([[aa_to_i[x] for x in seq] 
                    for seq in seqs])
    return X, aa_to_i
def one_hot_encode(s):
    return np.vstack([np.zeros(20), np.eye(20)])[s].flatten()

In [4]:
### get all available msa sequences
yeast_seq_str = []
fasta_sequences = SeqIO.parse(open("PABP_YEAST/data/PABP_YEAST.a2m"),'fasta')
for fasta in fasta_sequences:
    yeast_seq_str.append(str(fasta.seq))

In [5]:
def check_sequence(s, alphabet=ALPHABET_PROTEIN):
    for aa in s:
        if aa not in ALPHABET_PROTEIN:
            return False
    return True
def process_msa_sequence(msa_sequences):
    ''' takes in list of sequences and one-hot encodes the sequences'''
    pos_upper = [x for x in range(len(msa_sequences[0])) if not msa_sequences[0][x].islower()]
    msa_sequences = np.asarray([np.asarray(list(s))[pos_upper] for s in msa_sequences if not 'x' in s])
    msa_sequences = np.asarray([s for s in msa_sequences if check_sequence(s) and len(s)==82])
    msa_sequences = np.asarray(msa_sequences)

    seqs_enc, aa_to_i = encode(msa_sequences)
    oh_enc_seq = []
    for s in seqs_enc:
        oh_enc_seq.append(one_hot_encode(s))
    oh_enc_seq = np.asarray(oh_enc_seq)
    return oh_enc_seq

def valid_weights_from_model(c):
    ### returns only valid weights
    _w = c.weights
    _w_valid = []
    for i in range(c.weights.shape[0]):
        if _w[i] == 0: 
            continue
        _w_valid.append(1/_w[i])
    return _w_valid

In [6]:
processed = process_msa_sequence(yeast_seq_str)

In [7]:
c = CouplingsModel(f"PABP_YEAST/model/PABP_YEAST.model_params")
weights = valid_weights_from_model(c)

In [8]:
len(weights)

151496

In [9]:
len(processed)

151496

In [7]:
from sklearn.svm import OneClassSVM

clf = OneClassSVM(kernel='poly', degree=2, nu=0.3)
train = processed[:50000]
clf.fit(train, sample_weight=weights[:50000])

NameError: name 'weights' is not defined

In [11]:
pred = clf.predict(processed[50000:55000])

In [12]:
pred[np.where(pred==1)].shape

(2782,)

In [42]:
# using the trained svm model on DMS data
wildtype = yeast_seq_str[0]
data = pd.read_csv(
    "PABP_YEAST/data/PABP_YEAST_Fields2013-singles.csv", sep=";", comment="#"
)
mutant, label = data['mutant'].to_numpy(), data['linear'].to_numpy()
mutant_data = []
for m in mutant:
    original_aa, loc, mutant_aa = m[0], int(m[1:4])-115, m[4]
    assert wildtype[loc] == original_aa
    mutant_data.append(wildtype[:loc]+mutant_aa+wildtype[loc+1:])
mutant_data = np.asarray(mutant_data)
mutant_data = process_msa_sequence(mutant_data)
print(mutant_data.shape)

(1188, 1640)


In [43]:
mutant_pred = clf.predict(mutant_data)

NameError: name 'clf' is not defined

In [44]:
pred[np.where(mutant_pred==1)].shape

NameError: name 'pred' is not defined

In [45]:
for i in range(label.shape[0]):
    label[i] = 1 if label[i] > 0.7 else 0

In [46]:
label.shape

(1188,)

In [47]:
label[np.where(label==1)].shape

(673,)

In [48]:
import sklearn
clf2 = sklearn.svm.SVC(kernel='poly', degree=2)
split = mutant_data.shape[0]//5 * 4
mut_train = mutant_data[:split]
mut_test = mutant_data[split:]
clf2.fit(mut_train, label[:split])

SVC(degree=2, kernel='poly')

In [49]:
pred2 = clf2.predict(mut_test)

In [50]:
pred2

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.